# Finding iLo Mac address

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>find_ilo_mac_address!!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [ ]:
import sys
import json
import getpass
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

#from get_resource_directory import get_resource_directory


<div class="alert alert-block alert-success">
    <p>This is our find_ilo_mac_address function overview, in the end this will give us about finding ilo mac address related information.</p>
    <p>Let me give you a walk through of it!</p>
    <ol>
    <li> For the identification of a device, its mac address is very much required. If you have the Systems Management Homepage installed, you can find out by clicking "Management Processor" and then check the Physical Address field.</li>
    <li> If we don't have a resource directory, information about the ethernet data is stored inside the managers json object. Through managers_response variable we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods">GET operation</a> to and trying to obtain the virtual media about iLO system.</li>
        <li> Ethernet interface gives the response for that member about the <b>ilo mac address</b></li>
        <li> If the resource directory is present, then we will simply iterate through all instances inside resources and if the ethernet interface is present in <b>@odata.type</b> attribute then we will store information about ethernet interface in <b>ethernet_data</b> dictionary and break out of iteration.</li>
        <li> Finally, we get <b>iLo Network Manager Interface</b> and <b>iLO Mac Address</b> as output.</li>
    </ol>
</div>

In [ ]:
def find_ilo_mac_address(_redfishobj):

    ethernet_data = {}

    resource_instances = _redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        manager_ethernet_interfaces = managers_members_response.obj['EthernetInterfaces']\
                                                                                    ['@odata.id']
        manager_ethernet_interfaces_response = _redfishobj.get(manager_ethernet_interfaces)
        manager_ethernet_interfaces_members = manager_ethernet_interfaces_response.\
                                                            obj['Members']
        for _member in manager_ethernet_interfaces_members:
            _tmp = _redfishobj.get(_member['@odata.id']).obj
            ethernet_data[_member['@odata.id']] = _tmp
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#EthernetInterfaceCollection.' in instance['@odata.type'] and 'Managers' in \
                                                                        instance['@odata.id']:
                ethernet_uri = instance['@odata.id']
                ethernet_interfaces = _redfishobj.get(ethernet_uri).obj['Members']
                for _ethernet_interface in ethernet_interfaces:
                    ethernet_data[_ethernet_interface['@odata.id']] = _redfishobj.\
                                                        get(_ethernet_interface['@odata.id']).dict
    for iface in ethernet_data:
        sys.stdout.write("iLO Network Manager Interface: \'%s\'\n" % ethernet_data[iface]['Id'])
        sys.stdout.write("\tiLO Mac Address: \'%s\'\n" % ethernet_data[iface].get('MACAddress'))


<div class="alert alert-block alert-success">
<b> <h1> 3.</h1>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE.<br>
    <ul>
        <li>Once you enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will  be logged in to the server</li>
        <li>If server is down or Unreachable we get "server not reachable" message!</li>
        <li>and Once everything is done you are automatically logged out with <b>.logout()<b> 😁. </li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = input("Enter iLO IP address: ")
    LOGIN_ACCOUNT = input("Enter Username: ")
    LOGIN_PASSWORD = getpass.getpass("Enter password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    find_ilo_mac_address(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class="alert alert-block alert-success">
    In case you need help 🔗 links to relevant resources :<br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
</div>